In [149]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

%matplotlib inline
plt.rcParams["patch.force_edgecolor"] = True
sns.set() 

In [150]:
df = pd.read_excel('C:\\Users\\allen.tseng\\Desktop\\excel-comp-data.xlsx')
df.head()

,account,name,street,city,state,postal-code,Jan,Feb,Mar
0,211829,"Kerluke, Koepp and Hilpert",34456 Sean Highway,New Jaycob,Texas,28752,10000,62000,35000
1,320563,Walter-Trantow,1311 Alvis Tunnel,Port Khadijah,NorthCarolina,38365,95000,45000,35000
2,648336,"Bashirian, Kunde and Price",62184 Schamberger Underpass Apt. 231,New Lilianland,Iowa,76517,91000,120000,35000
3,109996,"D'Amore, Gleichner and Bode",155 Fadel Crescent Apt. 144,Hyattburgh,Maine,46021,45000,120000,10000
4,121213,Bauch-Goldner,7274 Marissa Common,Shanahanchester,California,49681,162000,120000,35000


In [151]:
df['Total'] = df['Jan'] + df['Feb'] + df['Mar']
df.head()

,account,name,street,city,state,postal-code,Jan,Feb,Mar,Total
0,211829,"Kerluke, Koepp and Hilpert",34456 Sean Highway,New Jaycob,Texas,28752,10000,62000,35000,107000
1,320563,Walter-Trantow,1311 Alvis Tunnel,Port Khadijah,NorthCarolina,38365,95000,45000,35000,175000
2,648336,"Bashirian, Kunde and Price",62184 Schamberger Underpass Apt. 231,New Lilianland,Iowa,76517,91000,120000,35000,246000
3,109996,"D'Amore, Gleichner and Bode",155 Fadel Crescent Apt. 144,Hyattburgh,Maine,46021,45000,120000,10000,175000
4,121213,Bauch-Goldner,7274 Marissa Common,Shanahanchester,California,49681,162000,120000,35000,317000


In [152]:
#get the sum of Jan
df['Jan'].sum()

1462000

In [153]:
#get the mean of Jan
df['Jan'].mean()

97466.66666666667

In [154]:
#get the min and max of Jan
df['Jan'].min() , df['Jan'].max()

(10000, 162000)

In [155]:
#get grand total
#get the sum of each month, sum them together
grand_total = df['Jan'].sum() + df['Feb'].sum() + df['Mar'].sum()
grand_total

3686000

In [156]:
#get month total and grand total
#get the sum of each column: Jan, Feb, Mar, Total
sum_row = df[['Jan','Feb','Mar','Total']].sum()
sum_row

Jan      1462000
Feb      1507000
Mar       717000
Total    3686000
dtype: int64

In [157]:
#Turn the series into a dataframe, transpose from rows to columns
df_sum = pd.DataFrame(data = sum_row).T
df_sum

,Jan,Feb,Mar,Total
0,1462000,1507000,717000,3686000


In [158]:
#add the missing columns before adding the totals back to original data frame
df_sum = df_sum.reindex(columns = df.columns)
df_sum

,account,name,street,city,state,postal-code,Jan,Feb,Mar,Total
0,NaN,NaN,NaN,NaN,NaN,NaN,1462000,1507000,717000,3686000


In [159]:
df_final = df.append(df_sum, ignore_index = True)
df_final.tail()

,account,name,street,city,state,postal-code,Jan,Feb,Mar,Total
11,231907.0,Hahn-Moore,18115 Olivine Throughway,Norbertomouth,NorthDakota,31415.0,150000,10000,162000,322000
12,242368.0,"Frami, Anderson and Donnelly",182 Bertie Road,East Davian,Iowa,72686.0,162000,120000,35000,317000
13,268755.0,Walsh-Haley,2624 Beatty Parkways,Goodwinmouth,RhodeIsland,31919.0,55000,120000,35000,210000
14,273274.0,McDermott PLC,8917 Bergstrom Meadow,Kathryneborough,Delaware,27933.0,150000,120000,70000,340000
15,NaN,NaN,NaN,NaN,NaN,NaN,1462000,1507000,717000,3686000


# Additional Data Transformation

In [160]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
state_to_code = {"VERMONT": "VT", "GEORGIA": "GA", "IOWA": "IA", "Armed Forces Pacific": "AP", "GUAM": "GU",
                 "KANSAS": "KS", "FLORIDA": "FL", "AMERICAN SAMOA": "AS", "NORTH CAROLINA": "NC", "HAWAII": "HI",
                 "NEW YORK": "NY", "CALIFORNIA": "CA", "ALABAMA": "AL", "IDAHO": "ID", "FEDERATED STATES OF MICRONESIA": "FM",
                 "Armed Forces Americas": "AA", "DELAWARE": "DE", "ALASKA": "AK", "ILLINOIS": "IL",
                 "Armed Forces Africa": "AE", "SOUTH DAKOTA": "SD", "CONNECTICUT": "CT", "MONTANA": "MT", "MASSACHUSETTS": "MA",
                 "PUERTO RICO": "PR", "Armed Forces Canada": "AE", "NEW HAMPSHIRE": "NH", "MARYLAND": "MD", "NEW MEXICO": "NM",
                 "MISSISSIPPI": "MS", "TENNESSEE": "TN", "PALAU": "PW", "COLORADO": "CO", "Armed Forces Middle East": "AE",
                 "NEW JERSEY": "NJ", "UTAH": "UT", "MICHIGAN": "MI", "WEST VIRGINIA": "WV", "WASHINGTON": "WA",
                 "MINNESOTA": "MN", "OREGON": "OR", "VIRGINIA": "VA", "VIRGIN ISLANDS": "VI", "MARSHALL ISLANDS": "MH",
                 "WYOMING": "WY", "OHIO": "OH", "SOUTH CAROLINA": "SC", "INDIANA": "IN", "NEVADA": "NV", "LOUISIANA": "LA",
                 "NORTHERN MARIANA ISLANDS": "MP", "NEBRASKA": "NE", "ARIZONA": "AZ", "WISCONSIN": "WI", "NORTH DAKOTA": "ND",
                 "Armed Forces Europe": "AE", "PENNSYLVANIA": "PA", "OKLAHOMA": "OK", "KENTUCKY": "KY", "RHODE ISLAND": "RI",
                 "DISTRICT OF COLUMBIA": "DC", "ARKANSAS": "AR", "MISSOURI": "MO", "TEXAS": "TX", "MAINE": "ME"}

In [145]:
process.extractOne("Minnesotta", choices = state_to_code.keys())

('MINNESOTA', 95)

In [146]:
process.extractOne("AlaBAMMazzz",choices=state_to_code.keys(),score_cutoff=78)

('ALABAMA', 78)

In [147]:
def convert_state(row):
    abbrev = process.extractOne(row["state"],choices=state_to_code.keys(),score_cutoff=80)
    if abbrev:
        return state_to_code[abbrev[0]]
    return np.nan

In [148]:
#insert a column named abbrev, fill the values with NaN
df_final.insert(6, "abbrev", np.nan)
df_final.head()

,account,name,street,city,state,postal-code,abbrev,Jan,Feb,Mar,Total
0,211829.0,"Kerluke, Koepp and Hilpert",34456 Sean Highway,New Jaycob,Texas,28752.0,NaN,10000,62000,35000,107000
1,320563.0,Walter-Trantow,1311 Alvis Tunnel,Port Khadijah,NorthCarolina,38365.0,NaN,95000,45000,35000,175000
2,648336.0,"Bashirian, Kunde and Price",62184 Schamberger Underpass Apt. 231,New Lilianland,Iowa,76517.0,NaN,91000,120000,35000,246000
3,109996.0,"D'Amore, Gleichner and Bode",155 Fadel Crescent Apt. 144,Hyattburgh,Maine,46021.0,NaN,45000,120000,10000,175000
4,121213.0,Bauch-Goldner,7274 Marissa Common,Shanahanchester,California,49681.0,NaN,162000,120000,35000,317000


In [118]:
#drop row 15 , where we calculated the month and grand totals
df_final = df_final.drop(index = 15)

In [119]:
#use apply to add the abbrevation
#df_final['abbrev'] = df_final.apply(lambda x: convert_state(x[state], axis = 1)
df_final['abbrev'] = df_final.apply(convert_state, axis=1)
df_final.tail()

,account,name,street,city,state,postal-code,abbrev,Jan,Feb,Mar,Total
10,214098.0,"Goodwin, Homenick and Jerde",649 Cierra Forks Apt. 078,Rosaberg,Tenessee,47743.0,TN,45000,120000,55000,220000
11,231907.0,Hahn-Moore,18115 Olivine Throughway,Norbertomouth,NorthDakota,31415.0,ND,150000,10000,162000,322000
12,242368.0,"Frami, Anderson and Donnelly",182 Bertie Road,East Davian,Iowa,72686.0,IA,162000,120000,35000,317000
13,268755.0,Walsh-Haley,2624 Beatty Parkways,Goodwinmouth,RhodeIsland,31919.0,RI,55000,120000,35000,210000
14,273274.0,McDermott PLC,8917 Bergstrom Meadow,Kathryneborough,Delaware,27933.0,DE,150000,120000,70000,340000


In [120]:
#summarize by state using the groupby function
df_sub = df_final[['abbrev','Jan','Feb','Mar','Total']].groupby('abbrev').sum()
df_sub

,Jan,Feb,Mar,Total
abbrev,,,,
AR,150000,120000,35000,305000
CA,162000,120000,35000,317000
DE,150000,120000,70000,340000
IA,253000,240000,70000,563000
ID,70000,120000,35000,225000
ME,45000,120000,10000,175000
MS,62000,120000,70000,252000
NC,95000,45000,35000,175000
ND,150000,10000,162000,322000


In [121]:
#pivot accomplishes the same thing, but the columns are sorted alphabetical order
df_pivot = df_final.pivot_table(values = ['Jan','Feb','Mar','Total'], index = 'abbrev', aggfunc = 'sum')
df_pivot

,Feb,Jan,Mar,Total
abbrev,,,,
AR,120000,150000,35000,305000
CA,120000,162000,35000,317000
DE,120000,150000,70000,340000
IA,240000,253000,70000,563000
ID,120000,70000,35000,225000
ME,120000,45000,10000,175000
MS,120000,62000,70000,252000
NC,45000,95000,35000,175000
ND,10000,150000,162000,322000


In [122]:
df_final[['abbrev','Jan','Feb','Mar','Total']]

,abbrev,Jan,Feb,Mar,Total
0,TX,10000,62000,35000,107000
1,NC,95000,45000,35000,175000
2,IA,91000,120000,35000,246000
3,ME,45000,120000,10000,175000
4,CA,162000,120000,35000,317000
5,AR,150000,120000,35000,305000
6,MS,62000,120000,70000,252000
7,RI,145000,95000,35000,275000
8,PA,70000,95000,35000,200000
9,ID,70000,120000,35000,225000


In [124]:
df_final[['abbrev','Jan','Feb','Mar','Total']].groupby('abbrev').sum()

,Jan,Feb,Mar,Total
abbrev,,,,
AR,150000,120000,35000,305000
CA,162000,120000,35000,317000
DE,150000,120000,70000,340000
IA,253000,240000,70000,563000
ID,70000,120000,35000,225000
ME,45000,120000,10000,175000
MS,62000,120000,70000,252000
NC,95000,45000,35000,175000
ND,150000,10000,162000,322000


In [128]:
#we want to format the data as currency by using applymap to all the values in the data frame
def money(x):
    return "${:,.0f}".format(x)

formatted_df = df_sub.applymap(money)
formatted_df

,Jan,Feb,Mar,Total
abbrev,,,,
AR,"$150,000","$120,000","$35,000","$305,000"
CA,"$162,000","$120,000","$35,000","$317,000"
DE,"$150,000","$120,000","$70,000","$340,000"
IA,"$253,000","$240,000","$70,000","$563,000"
ID,"$70,000","$120,000","$35,000","$225,000"
ME,"$45,000","$120,000","$10,000","$175,000"
MS,"$62,000","$120,000","$70,000","$252,000"
NC,"$95,000","$45,000","$35,000","$175,000"
ND,"$150,000","$10,000","$162,000","$322,000"


In [131]:
sum_row = df_sub[['Jan','Feb','Mar','Total']].sum()
sum_row

Jan      1462000
Feb      1507000
Mar       717000
Total    3686000
dtype: int64

In [132]:
df_sub_sum=pd.DataFrame(data=sum_row).T
df_sub_sum=df_sub_sum.applymap(money)
df_sub_sum

,Jan,Feb,Mar,Total
0,"$1,462,000","$1,507,000","$717,000","$3,686,000"
